In [1]:
from pyfmpcloud import settings
from pyfmpcloud import company_valuation as cv
from pyfmpcloud import stock_time_series as sts
import pandas as pd
import numpy as np
import numpy_financial as npf
from pandas_datareader import data as wb
import datetime
import requests
from scipy.stats.mstats import gmean
import statsmodels.api as sm

ModuleNotFoundError: ignored

In [2]:
start_time = datetime.datetime.now() # Замер времени выполнения скрипта для одного тикера

NameError: ignored

In [ ]:
COMPANY = 'PG'
KEY = '2105b9f242d47b69fc73f0f2205be048'

In [ ]:
IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{COMPANY}?apikey={KEY}').json()
BL = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{COMPANY}?apikey={KEY}').json()
RT = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{COMPANY}?apikey={KEY}').json()
CF = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{COMPANY}?apikey={KEY}').json()
EV = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{COMPANY}?apikey={KEY}').json()
PF = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{COMPANY}?apikey={KEY}').json()
CQ = requests.get(f'https://financialmodelingprep.com/api/v3/quote/{COMPANY}?apikey={KEY}').json()

In [ ]:
income_statement = pd.DataFrame(IS)
balance = pd.DataFrame(BL)
ratios = pd.DataFrame(RT)
cash_flow = pd.DataFrame(CF)
enterprise_value = pd.DataFrame(EV)
company_profile = pd.DataFrame(PF)
real_time_quote = pd.DataFrame(CQ)

In [ ]:
work_table = pd.concat((income_statement[['date', 'revenue', 'operatingIncome', 'netIncome', 'eps', 'ebitda', 'depreciationAndAmortization', 'interestExpense']].replace(np.nan, 0)[0:10], balance[['totalAssets', 'totalLiabilities', 'totalStockholdersEquity', 'totalDebt']].replace(np.nan, 0)[0:10]
                    ,ratios[['dividendPayoutRatio', 'dividendYield', 'effectiveTaxRate']].replace(np.nan, 0)[0:10],cash_flow[['freeCashFlow', 'dividendsPaid']].replace(np.nan, 0)[0:10], enterprise_value[['numberOfShares', 'stockPrice']].replace(np.nan, 0)[0:10] ),
                       axis = 1)

In [ ]:
LTGrowth = 0.02
PRICE = real_time_quote['price'][0]
company_profile_beta = company_profile['beta']
Total_rating = []

In [ ]:
print(work_table)

Расчет показателей качества

In [ ]:
# считаем рост выручки______________________________________________________________
revenue_growth = round((((work_table['revenue'][0]/work_table['revenue'][9]) ** 0.1) -1), 2)

print(f'revenue_growth {revenue_growth}')


if revenue_growth > 0.06:
    revenue_growth_rating = 1
elif 0.05 > revenue_growth > 0.02:
    revenue_growth_rating = 0
else:
    revenue_growth_rating = -1

Total_rating.append(revenue_growth_rating)

# считаем Использование прибыли______________________________________________________________
average_dividend_payout_percentage = np.mean(work_table['dividendYield'][0:])

# average_percentage_of_buyback  # не найден   !!!!!!!!!!!!!!!!!!!!!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!!!!!!!!!!!!!!!!!!!!!!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

SHAREHOLDERS_EQUITY = (work_table['totalAssets'] - work_table['totalLiabilities'])

average_quity_SHAREHOLDERS_EQUITY = round((((SHAREHOLDERS_EQUITY[0] / SHAREHOLDERS_EQUITY[9]) ** 0.1) -1), 2)

use_of_profit = round((average_dividend_payout_percentage + average_quity_SHAREHOLDERS_EQUITY), 2)

print(f'use_of_profit {use_of_profit}')

if use_of_profit < 0.06:
    use_of_profit_rating = -2
elif 0.1 < use_of_profit > 0.06:
    use_of_profit_rating = 0
else:
    use_of_profit_rating = 2

Total_rating.append(use_of_profit_rating)

# Считаем Операционную маржу_________________________________________________________________

OPERATING_MAR = work_table['operatingIncome']/work_table['revenue']

operating_margin = round(np.mean(OPERATING_MAR[0:]), 2)

print(f'operating_margin {operating_margin}')

if operating_margin < 0.06:
    operating_margin_rating = -2
elif 0.14 > operating_margin > 0.06:
    operating_margin_rating = 0
else:
    operating_margin_rating = 2

Total_rating.append(operating_margin_rating)

# Считаем Динамику операционной маржи_________________________________________________________________

if OPERATING_MAR.std() < 0.1:
    operating_margin_dynamics = 'Рост'
    operating_margin_dynamics_rating = 2
else:
    operating_margin_dynamics = 'Падение'
    operating_margin_dynamics_rating = -2

Total_rating.append(operating_margin_dynamics_rating)

print(f'operating_margin_dynamics {operating_margin_dynamics}')

# Считаем Маржу чистой прибыли_________________________________________________________________

NET_PROFIT_MAR = work_table['netIncome']/work_table['revenue']

net_profit_margin = round(np.mean(NET_PROFIT_MAR[0:]), 2)

print(f'net_profit_margin {net_profit_margin}')

if net_profit_margin < 0.06:
    net_profit_margin_rating = -2
elif 0.1 > net_profit_margin > 0.06:
    net_profit_margin_rating = 0
else:
    net_profit_margin_rating = 2

Total_rating.append(net_profit_margin_rating)

# Считаем Динамику маржи чистой прибыли_________________________________________________________________

if NET_PROFIT_MAR.std() < 0.1 :
    net_profit_margin_dynamics = 'Рост'
    net_profit_margin_dynamics_rating = 1
else:
    net_profit_margin_dynamics = 'Падение'
    net_profit_margin_dynamics_rating = -1

Total_rating.append(net_profit_margin_dynamics_rating)

print(f'net_profit_margin_dynamics {net_profit_margin_dynamics}')

# Считаем Рост EPS_________________________________________________________________

EPS_growth = round((((work_table['eps'][0]/work_table['eps'][9]) ** 0.1) -1), 2)   # Первый вариант, eps - берем из API

# EPS_growth = round((((SHAREHOLDERS_EQUITY[0] / SHAREHOLDERS_EQUITY[9]) ** 0.1) -1), 2)  # Второй вариант, eps = totalAssets - totalLiabilities

#  !!!!!!!!!!!!!!!!!При разных вариантах разные значения!!!!!!!!!!!!!!!!!!!!!!

print(f'EPS_growth {EPS_growth}')

if EPS_growth < 0.06:
    EPS_growth_rating = -2
elif 0.1 < EPS_growth > 0.06:
    EPS_growth_rating = 0
else:
    EPS_growth_rating = 2

Total_rating.append(EPS_growth_rating)

# Считаем Уровень долга_________________________________________________________________

debt_level = round(np.mean(work_table['totalLiabilities']/work_table['totalAssets']), 2)

print(f'debt_level {debt_level}')

if debt_level < 0.5:
    debt_level_rating = 1
elif 0.7 > debt_level > 0.5:
    debt_level_rating = 0
else:
    debt_level_rating = -1

Total_rating.append(debt_level_rating)

# Считаем ROE_________________________________________________________________

ROE_average = round(np.mean((work_table['netIncome']/work_table['revenue']) * (work_table['revenue']/work_table['totalAssets']) * \
      (work_table['totalAssets']/work_table['totalStockholdersEquity'])), 2)

print(f'ROE_average {ROE_average}')

if ROE_average < 0.08:
    ROE_average_rating = -2
elif 0.14 > ROE_average > 0.08:
    ROE_average_rating = 0
else:
    ROE_average_rating = 2

Total_rating.append(ROE_average_rating)

# Считаем ROA_________________________________________________________________

ROA_average = round(np.mean(work_table['netIncome']/work_table['totalAssets']), 2)

print(f'ROA_average {ROA_average}')

if ROA_average < 0.06:
    ROA_average_rating = -1
elif 0.08 > ROA_average > 0.06:
    ROA_average_rating = 0
else:
    ROA_average_rating = 1

Total_rating.append(ROA_average_rating)

# Получаем общую оценку_________________________________________________________________

print(Total_rating)
print(f'Общая оценка = {sum(Total_rating)}')

# Оцениваем инвестиционную стоимость  

In [ ]:
print('Считаем инвестиционную стоимость по дивидендной модели Гордона')

def valuecompany(quote):
    Dtoday =   work_table['dividendsPaid'][0] / work_table['numberOfShares'][0]

    # ROE and PAyout Ratio

    ROE_formul = ROE_average
    print(f'ROE_formul {ROE_formul}')

    payout_ratio = work_table['dividendPayoutRatio'][0]
    sustgrwothrate = ROE_formul * (1 - payout_ratio)

    ##Cost of equity:
    # Risk Free Rate
    start = datetime.datetime(2019, 2, 1)
    end = datetime.datetime(2020, 2, 27)

    Treasury = wb.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF / 100
    print(f'RF {RF}')

    # Beta
    beta = float(company_profile_beta)
    print(f'beta {beta}')

    # Market Return
    # Считаем годовую доходность
    data = wb.DataReader('^GSPC', data_source='yahoo', start='2001-1-1') # нужно определиться с переменной
    data['simple_return'] = (data['Adj Close'] / data['Adj Close'].shift(1)) - 1
    data['simple_return'].replace(np.nan, 0)
    avg_returns_a = data['simple_return'].mean() * 250

    print(f'Годовая доходность S&P500 {avg_returns_a}')

    # Drop all Not a number values using drop method.
    ke = RF + (beta * (avg_returns_a - RF))

    DDM = (Dtoday * (1 + sustgrwothrate)) / (ke - sustgrwothrate)

    print(f'Расчетная стоимость по модели Гордона {DDM}')
    return DDM

valuecompany(COMPANY)

print(f'Время выполнения :{datetime.datetime.now() - start_time}') # Замер времени

# Расчет методом дисконтирования FCF спрогнозируемого методом динейной регрессии

In [ ]:
# 'year_index' as the order of the year
work_table['year_index'] = range(1, len(work_table) + 1)[::-1]
# add intercept
work_table['intercept'] = 1

print(work_table)


# modeling
freeCashFlow_model = sm.OLS(work_table['freeCashFlow'], work_table[['year_index','intercept']]).fit()


freeCashFlow_pred_lr = freeCashFlow_model.predict([19,1])[0] #Прогнозируем через 10 лет


freeCashFlow_pred_lr5 = freeCashFlow_model.predict([14,1])[0] #Прогнозируем через 5 лет
freeCashFlow_pred_lr4 = freeCashFlow_model.predict([13,1])[0]
freeCashFlow_pred_lr3 = freeCashFlow_model.predict([12,1])[0]
freeCashFlow_pred_lr2 = freeCashFlow_model.predict([11,1])[0]
freeCashFlow_pred_lr1 = freeCashFlow_model.predict([10,1])[0]


# CAGR FCF

CF_forecast = {}

CF_forecast['next_year1'] = {}
CF_forecast['next_year1']['freeCashFlow'] = freeCashFlow_pred_lr1

CF_forecast['next_year2'] = {}
CF_forecast['next_year2']['freeCashFlow'] = freeCashFlow_pred_lr2

CF_forecast['next_year3'] = {}
CF_forecast['next_year3']['freeCashFlow'] = freeCashFlow_pred_lr3

CF_forecast['next_year4'] = {}
CF_forecast['next_year4']['freeCashFlow'] = freeCashFlow_pred_lr4

CF_forecast['next_year5'] = {}
CF_forecast['next_year5']['freeCashFlow'] = freeCashFlow_pred_lr5

print(CF_forecast)

print(f'freeCashFlow_pred_lr1 = {freeCashFlow_pred_lr1}')
print(f'freeCashFlow_pred_lr2 = {freeCashFlow_pred_lr2}')
print(f'freeCashFlow_pred_lr3 = {freeCashFlow_pred_lr3}')
print(f'freeCashFlow_pred_lr4 = {freeCashFlow_pred_lr4}')
print(f'freeCashFlow_pred_lr5 = {freeCashFlow_pred_lr5}')



print(freeCashFlow_pred_lr)


# ________________________________________  Расчет WACC  _______________________________________________


#Interest coverage ratio = EBIT / interest expenses

def interest_coveraga_and_RF(company):
  EBIT = work_table['ebitda'][0] - work_table['depreciationAndAmortization'][0]
  interest_expense = work_table['interestExpense'][0]
  interest_coverage_ratio = EBIT / interest_expense

    #RF
  start = datetime.datetime(1999, 12, 31)

  end= datetime.datetime.today().strftime('%Y-%m-%d')
  #end = datetime.datetime(2020, 7, 10)

  Treasury = wb.DataReader(['TB1YR'], 'fred', start, end)
  RF = float(Treasury.iloc[-1])
  RF = RF/100
  print(f'RF,interest_coverage_ratio = {RF,interest_coverage_ratio}')
  return [RF,interest_coverage_ratio]


#Cost of debt
def cost_of_debt(company, RF,interest_coverage_ratio):
  if interest_coverage_ratio > 8.5:
    #Rating is AAA
    credit_spread = 0.0063
  if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
    #Rating is AA
    credit_spread = 0.0078
  if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <=  6.5):
    #Rating is A+
    credit_spread = 0.0098
  if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <=  5.49):
    #Rating is A
    credit_spread = 0.0108
  if (interest_coverage_ratio > 3) & (interest_coverage_ratio <=  4.25):
    #Rating is A-
    credit_spread = 0.0122
  if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <=  3):
    #Rating is BBB
    credit_spread = 0.0156
  if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <=  2.5):
    #Rating is BB+
    credit_spread = 0.02
  if (interest_coverage_ratio > 2) & (interest_coverage_ratio <=  2.25):
    #Rating is BB
    credit_spread = 0.0240
  if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <=  2):
    #Rating is B+
    credit_spread = 0.0351
  if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <=  1.75):
    #Rating is B
    credit_spread = 0.0421
  if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <=  1.5):
    #Rating is B-
    credit_spread = 0.0515
  if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <=  1.25):
    #Rating is CCC
    credit_spread = 0.0820
  if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <=  0.8):
    #Rating is CC
    credit_spread = 0.0864
  if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <=  0.65):
    #Rating is C
    credit_spread = 0.1134
  if interest_coverage_ratio <=  0.2:
    #Rating is D
    credit_spread = 0.1512

  cost_of_debt = RF + credit_spread
  print(f'cost_of_debt = {cost_of_debt}')
  return cost_of_debt


def costofequity(company):

  #RF
  start = datetime.datetime(1999, 12, 31)
  end= datetime.datetime.today().strftime('%Y-%m-%d')
  #end = datetime.datetime(2020, 7, 10)

  Treasury = wb.DataReader(['TB1YR'], 'fred', start, end)
  Treasury.dropna(inplace = True)
  RF = float(Treasury.iloc[-1])
  RF = RF/100


    #Beta
  beta = float(company_profile_beta)


  #Market Return
  data = wb.DataReader('^GSPC', data_source='yahoo', start='2001-1-1') # нужно определиться с переменной
  data['simple_return'] = (data['Adj Close'] / data['Adj Close'].shift(1)) - 1
  data['simple_return'].replace(np.nan, 0)
  avg_returns_a = data['simple_return'].mean() * 250
  #start = datetime.datetime(1999, 12, 31)
  #end= datetime.datetime.today().strftime('%Y-%m-%d')

  #SP500 = wb.DataReader(['sp500'], 'fred', start, end)

#Drop all Not a number values using drop method.
  #SP500.dropna(inplace = True)

  #SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-252])-1

  cost_of_equity = RF+(beta*(avg_returns_a - RF))
  print(f'cost_of_equity = {cost_of_equity}')
  return cost_of_equity

#effective tax rate and capital structure
def wacc(company):

  ETR = work_table['effectiveTaxRate'][0]


  Debt_to = work_table['totalDebt'][0] / (work_table['totalDebt'][0] + work_table['totalStockholdersEquity'][0])
  equity_to = work_table['totalStockholdersEquity'][0] / (work_table['totalDebt'][0] + work_table['totalStockholdersEquity'][0])

  WACC = (kd*(1-ETR)*Debt_to) + (ke*equity_to)
  print(WACC,equity_to,Debt_to)
  print(f'WACC = {WACC}')
  return WACC


RF_and_IntCov = interest_coveraga_and_RF(COMPANY)
RF = RF_and_IntCov[0]
interest_coverage_ratio = RF_and_IntCov[1]
ke = costofequity(COMPANY)
kd = cost_of_debt(COMPANY, RF, interest_coverage_ratio)
wacc_company = wacc(COMPANY)
print('wacc of = ' + COMPANY + ' is \n' + str((wacc_company * 100)) + '%')


#FCF List of CFs for each year

CF_forec = pd.DataFrame.from_dict(CF_forecast,orient='columns')

FCF_List = CF_forec.iloc[0:].values.tolist()

# print(f'FCF_List = {FCF_List[0]}')


npv = npf.npv(wacc_company, FCF_List[0])
print(f'npv = {npv}')


Terminal_value = (freeCashFlow_pred_lr5 * (1 + LTGrowth)) /(wacc_company  - LTGrowth)

Terminal_value_Discounted = Terminal_value/( 1 + wacc_company) ** 4

print(f'Terminal_value_Discounted = {Terminal_value_Discounted}')

sum_num = npv + Terminal_value_Discounted
print(f'sum_num = {sum_num}')

spl_num = sum_num / work_table['numberOfShares'][0]
print(f'Estimated price = {spl_num}')

difference_model_and_market_value =  spl_num - PRICE
print(f'stock price = {PRICE}')

print(f'difference_model_and_market_value = {difference_model_and_market_value}')